## 引言

前文[欺诈文本分类微调（二）：生成正向数据集](https://golfxiao.blog.csdn.net/article/details/141274990)生成正向数据集的方法和过程，本文会使用一些真实会议的ASR文本来构建欺诈文本分类微调的反向数据集（非欺诈）。大概思路如下：
1. 读取所有的会议文本，过滤会议文本中的语气助词。
2. 大的发言段落二次切割成100字以内的小段落，用以和欺诈文本长度尽量一致。
3. 将切割后的小段落按照正向数据集的格式来组织，并保存对话集到文件，作为基础数据集供后续复用。



## 加载文件
定义要加载的asr文件所在目录以及有效asr的文件列表。

In [2]:
asr_dir = "/data2/anti_fraud/dataset/docs/"

file_names = [
    "广发煤炭_淮河能源近况交流电话会议.txt",
    "39046229_20221103_095950.txt", 
    "比亚迪专家会.stt",
    "五一座谈会.stt",
    "证券投资介绍咨询会.stt",
    "merge-summary-2ba2c48b-a1b1-434e-a877-58ece02c72e2.stt",
    "广发电新-锂电拐点系列一：电池排产环增趋势下材料采购价格再跟踪-2.txt",
    "专业术语-1.stt",
    "线下4月15日第一场网络会议.stt",
    "广发电新-锂电拐点系列一：电池排产环增趋势下材料采购价格再跟踪.txt",
    "广发商社-【清明消费观察】旅游专家交流.txt",
    "merge-summary-b92ee22a4d5c43ca8ee12ac7327568ecsum.stt",
    "广发化工-产业升级 材料突围系列五：国潮风起，汉服面料上游怎么看.txt",
    "广发交运-航运系列专题：以伊再生事端，中东危机是否会进一步蔓延.txt",
    "广发传媒-AI生成内容及AI大模型训练阶段版权问题讨论.txt",
    "广发商社-酒店行业专家电话会.txt",
    "广发军工-新·视角：低空经济产业谈.txt",
    "20240131新希望业绩预告解读.txt",
    "火电灵活性改造-财通电新_原文.txt",
    "merge-summary-e25f867df4d1487ebdf8426674d71fasum.stt", # 重复内容本身就会导致
    "merge-summary-ff3c24d2a88d48d3bffc1cef65b76cc9sum.stt",
    "广发商社|培育钻石行业专家交流.txt",
    "广发纺服|锦纶行业专家小范围交流.txt",
    "广发轻工|跨境系列专家会议——家居出海专家.txt",
    "广发机械|人形机器人系列二十二：电子皮肤：灵巧手优化的下一解.txt",
    "广发计算机|专家解读SEC批准美国现货比特币ETF.txt",
    "久远银海策略交流-原文科大讯飞.stt",
    "全面注册制改革对金融IT行业影响解读.stt",
    "商社顺周期沙龙.stt",
    "2023-09-21国寿资产会议录音-财通电新.stt",
    "火电灵活性改造-财通电新.stt",
    "9月12日财通-专家会议.stt",  
    "天风建筑建材｜公司深度路演之上海港湾.stt",
    "天风证券_周期&能源每周谈.stt",
    "生猪黑马之华统股份_203020.stt",
    "西南地产-春节后工地开复工调研数据解读_133031.stt",
    "西南军工_航发科技_20230905_140019.stt",
    "信达固收-瑞科转债线上调研_100058.stt",
    "信达食品&深业幸福家0~3岁育儿攻略第二讲：科学辅食添加_20.stt",
    "再论中报总结与展望——国君周期论剑电话会议_193217.stt",
    "中信广场西区-销售.stt",
    "海通证券-销售.stt",
    "美的置业_090019.stt",
    "中银证券-2023年2月金股电话会议_193114.stt",
    "全面注册制改革对金融IT行业影响解读.stt",
    "商社顺周期沙龙.stt",
    "西南轻工|玉马遮阳小范围交流_100023.stt",
    "国信证券-港股双周谈52_200012.stt",
    "浦银国际-新能源车型-比亚迪专家会_160021.stt",
    "浦银医药-医院眼科业务复苏情况_200029.stt",
    "农业深度报告巡礼.stt",
]

导入基础包，并复用[前文](https://golfxiao.blog.csdn.net/article/details/141274990)中的一些工具函数。

In [3]:
import os
import re
import textwrap
import pandas as pd
from typing import List

def filename(path):
    filename_with_ext = os.path.basename(path)
    filename, _ = os.path.splitext(filename_with_ext)
    return filename

def pretty_print(text):
    wrapped_text = textwrap.fill(text, width=80)  # 设定每行的最大字符数
    print(wrapped_text)

定义两个方法，功能分别如下：
- load_file：用于加载文件内容，并按"\n\n"切割成段落。
- filter_interjections：用于去掉段落中的语气助词，避免无效内容的干扰。

In [ ]:
def filter_interjections(content: str) -> str:
    interjections = ['啊','呃','呀','吧','唉','嘛','噢','嗯','喂','喽','喔','哦','呵','呸','嘿','嚯','嘞','哎','噫','哟','啧','咦']
    # 构建简化的正则表达式模式，直接将所有语气词连接在一起
    pattern = "|".join(interjections)
    # 使用re.sub()函数替换语气词为空字符串
    filtered_text = re.sub(pattern, "", content)
    return filtered_text

def load_file(file_path, filter_func=None):
    file_content = ""
    with open(file_path, "r", encoding='utf8') as f:
        file_content = f.read()
    contents = file_content.split('\n\n')
    
    if filter_func:
        contents = [filter_func(content) for content in contents]
    return contents

last_file_path = os.path.join(asr_dir, file_names[-1])
contents = load_file(last_file_path, filter_interjections)
for item in contents[6:10]: pretty_print(item) 

## 长段落切割

由于我们收集的所有会议ASR文件中的段落都是按照`[时间戳] [发言人]: [发言内容]`的格式组织的，这里定义一个段落格式解析函数，用于从段落中解析出发言人和发言内容。

In [ ]:
def parse_paragraph(text: str):
    paragraph_pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}) ([^:]+):(.*)"
    match = re.match(paragraph_pattern, text.strip())
    if not match:
        print(f"not match paragraph pattern: {text}")
        return "", "", ""
    return match.group(1), match.group(2), match.group(3).strip()

timestamp, speaker, content = parse_paragraph(contents[6])
timestamp, speaker, content, len(content) 

> 得到段落的发言内容后，有些发言内容比较长，就像上面示例的这个段落有500多字，而前文用gpt生成的单条对话内容普遍在100字左右，最长也就150字。为了避免内容长度成为区分正反向数据集的特征，我们有必要将发言内容切割，尽可能让正、反向的发言长度保持一致。

首先定义一个按照指定标点符号来对文本进行切割的函数，使用一句话的结束符`。？!`来切割段落。
> 在正则表达式中，前面的`^[{punctuation_marks}]*`用于匹配不包含结束符的正常文本，后面的`[{punctuation_marks}]`用于匹配结束符，把两者合起来就能捕获完整的一句话。

In [ ]:
# 提取标点符号为一个字符串变量
punctuation_marks = '。？！.?!'

# 使用变量构建正则表达式模式，如同普通字段串前面加`f`一样，允许在字符串中嵌入{}包裹的表达式。
pattern = rf'([^{punctuation_marks}]*[{punctuation_marks}])'

# 预编译正则表达式，大量使用时可以提高性能
regex = re.compile(pattern, re.UNICODE)

def split_text_with_punct(text):
    # 使用finditer查找所有匹配项
    matches = list(regex.finditer(text))
    # 提取匹配到的段落
    paragraphs = [match.group(0).strip() for match in matches if match.group(0).strip()]
    # 找到最后一个匹配的结束位置
    last_match_end = matches[-1].end() if paragraphs else 0
    # 如果最后一个匹配的结束位置不是文本的结尾，即段落未以结束符结束，则将剩余文本作为单独一段
    if last_match_end < len(text):
        remaining_text = text[last_match_end:].strip()
        if remaining_text:
            paragraphs.append(remaining_text)
    
    return paragraphs

split_text_with_punct(content)

定义切割长段落的函数，支持指定最小长度和最大长度来切割。其中，最小长度用于过滤太短的无效文本，最大长度用于切割长段落内容。

> 过滤太短文本的场景：语音转文字时，由于一些噪音的影响，会出现内容只有两三个字的无效段落，例如：'啊','任何','就是'，这些内容给到模型训练是无意义的，可能还会干扰模型对正常内容的学习。

In [ ]:
def split_long_paragraph(text, min_length=0, max_length=100):
    # 先分割文本
    sentences = split_text_with_punct(text)
    
    # 合并分割段落以满足长度要求
    result = []
    current_paragraph = ''
    for s in sentences:
        if len(current_paragraph) + len(s) <= max_length:  
            current_paragraph += s  
        else:
            # 如果当前段落过长，则将其添加到结果中，并开始新段落
            if len(current_paragraph) >= min_length:
                result.append(current_paragraph)
            current_paragraph = s
    
    # 添加最后一个段落
    if len(current_paragraph) >= min_length:
        result.append(current_paragraph)
    
    return result

split_long_paragraph(content, 5, 100)

上面是针对单个段落的切割，而实际场景中我们需要处理的是一个文件中的所有段落，而下面的`paragraphs_to_dataset`就用于将一批原始段落切割后以json对话集的格式返回。

In [ ]:
def paragraphs_to_dialogs(paragraphs):
    dataset = []
    for text in paragraphs:
        _, speaker, content = parse_paragraph(text)
        if content == "":
            continue
        small_paragraphs = split_long_paragraph(content, min_length=5, max_length=100)

        dialogs = [{'speaker': speaker, 'content':child_text, 'is_fraud': False} for child_text in small_paragraphs]
        dataset.extend(dialogs)
    return dataset

paragraphs_to_dialogs(contents)

## 主流程

写文件操作复用[前文](https://golfxiao.blog.csdn.net/article/details/141274990)定义的几个工具函数。

In [20]:
def build_to_rows_func(case_prefix=""):
    def to_rows(dialog: list):
        rows = []
        for item in dialog:
            data = {'case': case_prefix}
            for k in ['speaker', 'content', 'is_fraud']:
                data[k] = item[k]
            rows.append(data)
            
        return rows

    return to_rows

def build_write_func(file_name):
    def write(rows, header=False):
        df_new = pd.DataFrame(rows, index=range(len(rows)))
        df_new.to_csv(file_name, mode='a', header=header, index=False)
    return write

generate_dialogs用于为一个文件生成对话集，此处不再需要依赖GPT，直接将切割好的小段落按照指定的格式批量写入目标文件即可。

In [21]:
def generate_dialogs(file_path, output_path):
    texts = load_file(file_path, filter_interjections)
    dataset = paragraphs_to_dialogs(texts)
   
    write_fn = build_write_func(output_path)
    to_rows_fn = build_to_rows_func(filename(file_path))
    write_fn(to_rows_fn(dataset))
    
    print(f"{filename(file_path)}:write dialog count: {len(dataset)}")

反向数据集就不再区分那么多文件，处理后的对话集全部写入一个文件，在开始主循环之前先写入csv的列头。

In [ ]:
# 目标文件
output_path = f"../dataset/fraud/csv_dialogs/meeting_label_false_test.csv"

# 写header
header_df = pd.DataFrame(columns = ['case', 'speaker', 'content', 'is_fraud'])
header_df.to_csv(output_path, mode='w', header=True, index=False)

# 主循环
for fname in file_names[-1:]:
    file_path = os.path.join(asr_dir, fname)
    if not os.path.exists(file_path):
        print(f"file_path: {file_path} not exists")
        continue
    print(f"generate for file_path: {file_path} ...")
    generate_dialogs(file_path, output_path)


反向数据全部是本地处理，生成速度很快，结果预览如下：

In [ ]:
pd.read_csv(output_path).head()

以上就是反向数据集的生成过程，主要以语音转文字的ASR文件为基础，通过长度对齐和格式转换来处理成和正向数据集相同格式的对话集。

## 延伸阅读
- [欺诈文本分类微调（一）：基座模型选型](https://golfxiao.blog.csdn.net/article/details/141168571)
- [欺诈文本分类微调（二）：生成正向数据集](https://golfxiao.blog.csdn.net/article/details/141274990)